# Import 

In [1]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read

In [2]:

train = pl.read_csv('../data/input/train_ver2.csv', null_values=[' NA', 'NA','     NA', 'P'])
test = pl.read_csv('../data/input/test_ver2.csv', null_values=[' NA', 'NA','     NA'])

In [3]:
# # カラムのデータ型に不安があるときはpandasのほうが便利
# # DtypeWarningでカラム列を指定してくれた上でとりあえず読み込んでくれる

# trn = pd.read_csv('../data/input/train_ver2.csv')

# print(trn.iloc[:,[5,8,11,15]].columns)# mixed types columns

In [4]:
# # 試しにconyuempの中身を確認

# print(trn.conyuemp.unique())

# trn.groupby('conyuemp')['ncodpers'].count()

# 学習データ

In [5]:
print(train.shape)
train.head(3)

(13647309, 48)


fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
str,i64,str,str,str,i64,str,i64,i64,i64,str,f64,str,str,str,str,str,str,i64,i64,str,i64,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""2015-01-28""",1375586,"""N""","""ES""","""H""",35,"""2015-01-12""",0,6,1,null,1.0,"""A""","""S""","""N""",null,"""KHL""","""N""",1,29,"""MALAGA""",1,87218.1,"""02 - PARTICULA...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2015-01-28""",1050611,"""N""","""ES""","""V""",23,"""2012-08-10""",0,35,1,null,1.0,"""I""","""S""","""S""",null,"""KHE""","""N""",1,13,"""CIUDAD REAL""",0,35548.74,"""03 - UNIVERSIT...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2015-01-28""",1050612,"""N""","""ES""","""V""",23,"""2012-08-10""",0,35,1,null,1.0,"""I""","""S""","""N""",null,"""KHE""","""N""",1,13,"""CIUDAD REAL""",0,122179.11,"""03 - UNIVERSIT...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
print(train.glimpse())

Rows: 13647309
Columns: 48
$ fecha_dato            <str> 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28
$ ncodpers              <i64> 1375586, 1050611, 1050612, 1050613, 1050614, 1050615, 1050616, 1050617, 1050619, 1050620
$ ind_empleado          <str> N, N, N, N, N, N, N, N, N, N
$ pais_residencia       <str> ES, ES, ES, ES, ES, ES, ES, ES, ES, ES
$ sexo                  <str> H, V, V, H, V, H, H, H, H, H
$ age                   <i64> 35, 23, 23, 22, 23, 23, 23, 23, 24, 23
$ fecha_alta            <str> 2015-01-12, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10
$ ind_nuevo             <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ antiguedad            <i64> 6, 35, 35, 35, 35, 35, 35, 35, 35, 35
$ indrel                <i64> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
$ ult_fec_cli_1t        <str> None, None, None, None, None, None, None, None, None, None
$ indrel_1mes           <

In [7]:
num_col = [col for col in train.columns[:24] if str(train[col].dtype) in ['Int64','Float64']]
print(num_col)

cats_col = [col for col in train.columns[:24] if str(train[col].dtype) in ['Utf8']] # str型はUtf8で表現される
print(cats_col)

['ncodpers', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tipodom', 'cod_prov', 'ind_actividad_cliente', 'renta']
['fecha_dato', 'ind_empleado', 'pais_residencia', 'sexo', 'fecha_alta', 'ult_fec_cli_1t', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'segmento']


In [8]:
train.select(num_col).describe()

describe,ncodpers,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tipodom,cod_prov,ind_actividad_cliente,renta
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7
"""null_count""",0.0,27734.0,27734.0,27734.0,27734.0,150655.0,27735.0,93591.0,27734.0,2.794375e6
"""mean""",834904.211501,40.181465,0.059562,76.591941,1.178399,1.00081,1.0,26.57147,0.457811,134254.318238
"""std""",431565.025784,17.185087,0.236673,1671.806547,4.177469,0.039877,0.0,12.784017,0.498217,230620.238822
"""min""",15889.0,2.0,0.0,-9.99999e5,1.0,1.0,1.0,1.0,0.0,1202.73
"""max""",1.553689e6,164.0,1.0,256.0,99.0,4.0,1.0,52.0,1.0,2.8894e7
"""median""",931893.0,39.0,0.0,50.0,1.0,1.0,1.0,28.0,0.0,101850.0


In [9]:
train.select(cats_col).describe()

describe,fecha_dato,ind_empleado,pais_residencia,sexo,fecha_alta,ult_fec_cli_1t,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,segmento
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309"""
"""null_count""","""0""","""27734""","""27734""","""27804""","""27734""","""13622516""","""154437""","""27734""","""27734""","""13645501""","""186126""","""27734""","""0""","""189368"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""2015-01-28""","""A""","""AD""","""H""","""1995-01-16""","""2015-07-01""","""A""","""N""","""N""","""N""","""004""","""N""","""""","""01 - TOP"""
"""max""","""2016-05-28""","""S""","""ZW""","""V""","""2016-05-31""","""2016-05-30""","""R""","""S""","""S""","""S""","""RED""","""S""","""ZARAGOZA""","""03 - UNIVERSIT..."
"""median""",null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [38]:
# カテゴリー値の固有値を直接出力

for col in cats_col:
    uniq = train.select(col).get_columns()[0].unique().to_numpy()
    print('-' * 50)
    print('# col {}, n_uniq {}, uniq {}'.format(col, len(uniq), uniq))

--------------------------------------------------
# col fecha_dato, n_uniq 17, uniq ['2016-05-28' '2015-12-28' '2016-02-28' '2015-03-28' '2015-04-28'
 '2015-09-28' '2016-01-28' '2015-07-28' '2015-08-28' '2015-10-28'
 '2015-01-28' '2016-04-28' '2016-03-28' '2015-11-28' '2015-06-28'
 '2015-02-28' '2015-05-28']
--------------------------------------------------
# col ind_empleado, n_uniq 6, uniq ['F' None 'S' 'N' 'B' 'A']
--------------------------------------------------
# col pais_residencia, n_uniq 119, uniq ['ZW' 'ES' 'CO' 'MX' 'BR' 'VN' 'GA' 'KR' 'BE' 'SL' 'AU' 'FI' 'SV' 'EC'
 'CG' 'TG' 'TN' 'PR' 'PY' 'SG' 'RO' 'ML' 'GT' 'CA' 'DO' 'HR' 'MR' 'LY'
 'MK' 'NI' 'CN' 'CR' 'PK' 'EE' 'MD' 'ET' 'PL' 'AE' 'TR' 'HU' 'IT' 'AR'
 'SE' 'MA' 'LV' 'MM' 'KZ' 'GR' 'FR' 'IS' 'UY' 'IL' 'RU' 'UA' 'OM' 'SK'
 'AO' 'AT' 'CL' 'HK' 'ZA' 'LT' 'CF' 'KE' 'AL' 'PA' 'NO' 'TW' 'US' 'BM'
 'DZ' 'LU' 'DE' 'CM' 'MT' 'BZ' 'PT' 'LB' 'CZ' 'CD' 'JP' 'KW' 'MZ' 'TH'
 'BO' None 'HN' 'SN' 'SA' 'DJ' 'IE' 'BG' 'BA' 'VE' 'EG' 'NL

# テストデータ

In [39]:
print(test.shape)
test.head(3)

(929615, 24)


fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
str,i64,str,str,str,i64,str,i64,i64,i64,str,f64,str,str,str,str,str,str,i64,i64,str,i64,str,str
"""2016-06-28""",15889,"""F""","""ES""","""V""",56,"""1995-01-16""",0,256,1,null,1.0,"""A""","""S""","""N""","""N""","""KAT""","""N""",1,28,"""MADRID""",1,""" 326124.90""","""01 - TOP"""
"""2016-06-28""",1170544,"""N""","""ES""","""H""",36,"""2013-08-28""",0,34,1,null,1.0,"""I""","""S""","""N""",null,"""KAT""","""N""",1,3,"""ALICANTE""",0,""" NA""","""02 - PARTICULA..."
"""2016-06-28""",1170545,"""N""","""ES""","""V""",22,"""2013-08-28""",0,34,1,null,1.0,"""A""","""S""","""N""",null,"""KHE""","""N""",1,15,"""CORUÑA, A""",1,""" NA""","""03 - UNIVERSIT..."


In [40]:
print(test.glimpse())

Rows: 929615
Columns: 24
$ fecha_dato            <str> 2016-06-28, 2016-06-28, 2016-06-28, 2016-06-28, 2016-06-28, 2016-06-28, 2016-06-28, 2016-06-28, 2016-06-28, 2016-06-28
$ ncodpers              <i64> 15889, 1170544, 1170545, 1170547, 1170548, 1170550, 1170552, 1170553, 1170555, 1170557
$ ind_empleado          <str> F, N, N, N, N, N, N, N, N, N
$ pais_residencia       <str> ES, ES, ES, ES, ES, ES, ES, ES, ES, ES
$ sexo                  <str> V, H, V, H, H, V, H, H, V, H
$ age                   <i64> 56, 36, 22, 22, 22, 22, 51, 22, 22, 22
$ fecha_alta            <str> 1995-01-16, 2013-08-28, 2013-08-28, 2013-08-28, 2013-08-28, 2013-08-28, 2013-08-28, 2013-08-28, 2013-08-28, 2013-08-28
$ ind_nuevo             <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ antiguedad            <i64> 256, 34, 34, 34, 34, 34, 34, 34, 34, 34
$ indrel                <i64> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
$ ult_fec_cli_1t        <str> None, None, None, None, None, None, None, None, None, None
$ indrel_1mes           <f6

In [41]:
num_col = [col for col in test.columns[:24] if str(test[col].dtype) in ['Int64','Float64']]
print(num_col)

cats_col = [col for col in test.columns[:24] if str(test[col].dtype) in ['Utf8']] # str型はUtf8で表現される
print(cats_col)

['ncodpers', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tipodom', 'cod_prov', 'ind_actividad_cliente']
['fecha_dato', 'ind_empleado', 'pais_residencia', 'sexo', 'fecha_alta', 'ult_fec_cli_1t', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'renta', 'segmento']


In [42]:
test.select(num_col).describe()

describe,ncodpers,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tipodom,cod_prov,ind_actividad_cliente
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",929615.0,929615.0,929615.0,929615.0,929615.0,929615.0,929615.0,929615.0,929615.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,23.0,0.0,3996.0,0.0
"""mean""",879456.621092,40.249821,0.027849,77.728413,1.177422,1.000058,1.0,26.54615,0.425272
"""std""",448156.939785,17.185119,0.164541,1797.824518,4.166039,0.010779,0.0,12.836614,0.494384
"""min""",15889.0,2.0,0.0,-9.99999e5,1.0,1.0,1.0,1.0,0.0
"""max""",1.553689e6,164.0,1.0,257.0,99.0,3.0,1.0,52.0,1.0
"""median""",966425.0,39.0,0.0,55.0,1.0,1.0,1.0,28.0,0.0


In [43]:
test.select(cats_col).describe()

describe,fecha_dato,ind_empleado,pais_residencia,sexo,fecha_alta,ult_fec_cli_1t,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,renta,segmento
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615""","""929615"""
"""null_count""","""0""","""0""","""0""","""5""","""0""","""927932""","""23""","""0""","""0""","""929511""","""2081""","""0""","""0""","""0""","""2248"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""2016-06-28""","""A""","""AD""","""H""","""1995-01-16""","""2016-06-01""","""A""","""N""","""N""","""N""","""004""","""N""","""""",""" NA""","""01 - TOP"""
"""max""","""2016-06-28""","""S""","""ZW""","""V""","""2016-05-31""","""2016-06-29""","""P""","""S""","""S""","""S""","""RED""","""S""","""ZARAGOZA""","""28894395.51""","""03 - UNIVERSIT..."
"""median""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [44]:
# カテゴリー値の固有値を直接出力

for col in cats_col:
    uniq = test.select(col).get_columns()[0].unique().to_numpy()
    print('-' * 50)
    print('# col {}, n_uniq {}, uniq {}'.format(col, len(uniq), uniq))

--------------------------------------------------
# col fecha_dato, n_uniq 1, uniq ['2016-06-28']
--------------------------------------------------
# col ind_empleado, n_uniq 5, uniq ['A' 'N' 'B' 'F' 'S']
--------------------------------------------------
# col pais_residencia, n_uniq 118, uniq ['EE' 'VE' 'PK' 'RU' 'CF' 'PL' 'MM' 'GE' 'AR' 'CN' 'SK' 'MZ' 'HK' 'SA'
 'RS' 'SE' 'IT' 'IE' 'EG' 'KR' 'MX' 'TW' 'TG' 'PT' 'LY' 'GQ' 'GM' 'NL'
 'GI' 'EC' 'FI' 'NI' 'BO' 'GA' 'RO' 'SN' 'PA' 'PH' 'ES' 'IN' 'LB' 'UA'
 'ZA' 'AL' 'GW' 'QA' 'HU' 'US' 'GN' 'DJ' 'BE' 'KE' 'DE' 'CA' 'BM' 'CO'
 'KW' 'PE' 'IS' 'PY' 'OM' 'IL' 'DO' 'NG' 'GR' 'CM' 'MK' 'CD' 'BG' 'DZ'
 'HR' 'CI' 'MT' 'LV' 'KH' 'CG' 'MA' 'MR' 'CR' 'BR' 'NO' 'FR' 'ET' 'LT'
 'KZ' 'TH' 'CH' 'VN' 'SL' 'AU' 'CZ' 'AE' 'GB' 'DK' 'TN' 'ML' 'BZ' 'TR'
 'SV' 'CU' 'GT' 'GH' 'BY' 'AD' 'MD' 'LU' 'JM' 'PR' 'JP' 'AT' 'AO' 'NZ'
 'ZW' 'BA' 'CL' 'UY' 'SG' 'HN']
--------------------------------------------------
# col sexo, n_uniq 3, uniq ['V' 'H' None]
---------